<a href="https://colab.research.google.com/github/shivansh11235/Emotion-Detection-System-using-Pretrained-Model/blob/main/Facial_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface moviepy pytube opencv-python-headless librosa matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=7c968393c834f36c6b950cadb605bf6bed75795de96be08f0f8e8a3138385ba0
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Interviewer Technique - Getting it right.mp4 to Interviewer Technique - Getting it right.mp4


In [ ]:
import os
# Renamed the uploaded video to "interview.mp4"
uploaded_filename = list(uploaded.keys())[0]
os.rename(uploaded_filename, "interview.mp4")

In [ ]:
import cv2
import os

cap = cv2.VideoCapture("interview.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
count, frame_id = 0, 0
os.makedirs("frames", exist_ok=True)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_id % int(fps * 2) == 0:  # every 2 seconds
        cv2.imwrite(f"frames/frame{count}.jpg", frame)
        count += 1
    frame_id += 1
cap.release()

In [ ]:
from deepface import DeepFace
import os
def analyze_saved_frames(folder="frames"):
    emotions = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(".jpg"):
            filepath = os.path.join(folder, filename)
            try:
                result = DeepFace.analyze(img_path=filepath, actions=['emotion'], enforce_detection=False)
                emotions.append(result[0]['dominant_emotion'])
            except:
                emotions.append("undetected")
    return emotions

25-07-14 17:26:13 - Directory /root/.deepface has been created
25-07-14 17:26:13 - Directory /root/.deepface/weights has been created


In [ ]:
emotions = analyze_saved_frames("frames")


25-07-14 17:27:35 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 69.9MB/s]


In [ ]:
from collections import Counter

def summarize_emotions(emotion_list):
    filtered = [e for e in emotion_list if e != "undetected"]
    count = Counter(filtered)
    if not count:
        return "No faces detected.", {}
    dominant = count.most_common(1)[0][0]
    return dominant, dict(count)

dominant_emotion, emotion_counts = summarize_emotions(emotions)
print("Dominant Emotion:", dominant_emotion)
print("Emotion Breakdown:", emotion_counts)

Dominant Emotion: fear
Emotion Breakdown: {'angry': 19, 'happy': 19, 'fear': 44, 'neutral': 15, 'sad': 36}


In [ ]:
from moviepy.editor import VideoFileClip

def extract_audio(video_path, audio_out='audio.wav'):
    clip = VideoFileClip(video_path)
    clip.audio.write_audiofile(audio_out)
    return audio_out

audio_path = extract_audio("interview.mp4")
print(f"Audio extracted to: {audio_path}")

MoviePy - Writing audio in audio.wav


MoviePy - Done.
Audio extracted to: audio.wav


In [ ]:
import librosa
import numpy as np
def analyze_audio(audio_path):
    y, sr = librosa.load(audio_path)
    energy = np.square(y).mean()
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    pitch = librosa.yin(y, fmin=50, fmax=300)
    if energy < 0.01:
        mood = "sad"
    elif tempo < 90:
        mood = "nervous or low energy"
    elif pitch.mean() > 200:
        mood = "anxious or excited"
    else:
        mood = "calm or confident"

    return {
        "energy": energy,
        "tempo": tempo,
        "pitch_mean": pitch.mean(),
        "mood": mood
    }

audio_features = analyze_audio(audio_path)
print("Audio mood:", audio_features['mood'])

Audio mood: sad


In [ ]:
def final_interview_verdict(facial_emotion, audio_mood):
    if audio_mood == "sad":
        return "The candidate appeared Sad both vocally and facially."
    if audio_mood in ["nervous or low energy", "anxious or excited"]:
        if facial_emotion in ["fear", "sad", "angry"]:
            return "The candidate appeared Nervous and Anxious during the interview."
        else:
            return "The candidate seemed vocally nervous but facial expressions were neutral."
    if audio_mood == "calm or confident":
        if facial_emotion in ["happy", "neutral"]:
            return "The candidate appeared Confident and Happy."
        if facial_emotion in ["fear", "sad", "angry"]:
            return "The candidate sounded calm but facial expressions showed stress."
        return "The candidate appeared Calm but showed mixed emotions."
    return "Mixed or unclear emotional signals detected."

verdict = final_interview_verdict(dominant_emotion, audio_features['mood'])
print("\n🎯 Final Interview Verdict:")
print(verdict)


🎯 Final Interview Verdict:
The candidate appeared Sad both vocally and facially.
